---

## **Problem Statement**

In modern AI-driven applications, it is often necessary to enhance an agent's reasoning by giving it access to real-world information (from the internet) and computational capabilities (like solving math or science problems). Large Language Models like GPT-4 alone cannot access real-time data or perform complex computations. To build a truly intelligent assistant, we need to **augment the model with tools** that bridge these gaps.

---

## **Objectives**

The goal of this application is to:

1. Build a tool-using AI agent that can:
   - Search the web for current news or facts using **Google Search via SerpAPI**
   - Solve mathematical or factual queries using **Wolfram Alpha**
2. Integrate the above tools with **GPT-4 Turbo** using **LangChain’s latest `create_tool_calling_agent` API**
3. Maintain clean, modular, and fully compatible code using the **latest LangChain best practices**
4. Run example queries to demonstrate how the agent utilizes different tools based on the query intent

---

## **Outcomes**

- A functional, intelligent assistant capable of web search and symbolic computation.
- Tool usage is decided dynamically by the agent depending on the user’s query.
- Fully integrated with `.env` file for secure API key storage.
- The agent is extensible for further tools (e.g., weather, finance APIs, etc.).

---

In [ ]:
!pip install openai langchain langchain-community python-dotenv -q
# langchain-community – Community-maintained third-party tool integrations for LangChain. Community-maintained integrations like SerpAPI, Wolfram Alpha

In [ ]:
!pip install google-search-results -q

In [ ]:
!pip install wolframalpha -q

In [ ]:
import os
from dotenv import load_dotenv

# LangChain core components
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import Tool

# External API utilities from LangChain Community
from langchain_community.utilities.serpapi import SerpAPIWrapper
# SerpAPIWrapper	Access Google Search via SerpAPI

from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
# WolframAlphaAPIWrapper	Send queries to Wolfram Alpha for math, science, and factual questions

In [ ]:
# Step 1: Load API keys from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")
WOLFRAM_ALPHA_APPID = os.getenv("WOLFRAM_ALPHA_APPID")

In [ ]:
# Step 2: Initialize the LLM (GPT-4 Turbo) using OpenAI key
llm = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.3,
    api_key=OPENAI_API_KEY
)

In [ ]:
# Step 3: Set up SerpAPI (Google Search wrapper)
search_tool = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)

In [ ]:
# Step 4: Set up Wolfram Alpha wrapper (use for facts, math, science)
wolfram_tool = WolframAlphaAPIWrapper(wolfram_alpha_appid=WOLFRAM_ALPHA_APPID)

In [ ]:
# Step 5: Wrap both tools as LangChain Tools
tools = [
    Tool.from_function( # Converts a regular Python function (like .run()) into a LangChain-compatible tool.
        func=search_tool.run,
        name="google_search",
        description="Use this to search the internet for current events, recent information, or news."
    ),
    Tool.from_function(
        func=wolfram_tool.run,
        name="wolfram_alpha",
        description="Use this to answer math problems, scientific facts, or general knowledge questions."
    )
]

In [ ]:
# Step 6: Define prompt template (agent_scratchpad is required)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a smart assistant that uses tools like Google Search and Wolfram Alpha."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")  # Required for tool-using agents
])

In [ ]:
# Step 7: Create the tool-calling agent
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

In [ ]:
# Step 8: Wrap agent in an executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [ ]:
# Step 9: Run a query through the agent
# def run_agent(query: str):
#     print(f"\nUser: {query}")
#     result = agent_executor.invoke({"input": query})
#     print("Agent:", result["output"])

import asyncio
async def run_agent_async(query: str):
    print(f"\nUser: {query}")
    result = await agent_executor.ainvoke({"input": query})
    print("Agent:", result["output"])

In [ ]:
await run_agent_async("What is the square root of 4563?")


User: What is the square root of 4563?


> Entering new AgentExecutor chain...

Invoking: `wolfram_alpha` with `square root of 4563`


Assumption: sqrt(4563) 
Answer: 39 sqrt(3)The square root of 4563 is approximately \( 39 \sqrt{3} \).

> Finished chain.
Agent: The square root of 4563 is approximately \( 39 \sqrt{3} \).


In [ ]:
await run_agent_async("Who is the CEO of Twitter as of today?")


User: Who is the CEO of Twitter as of today?


> Entering new AgentExecutor chain...

Invoking: `google_search` with `current CEO of Twitter 2023`


Linda YaccarinoThe CEO of Twitter as of today is Linda Yaccarino.

> Finished chain.
Agent: The CEO of Twitter as of today is Linda Yaccarino.


In [ ]:
await run_agent_async("What is Operation Sindoor?")

In [ ]:
await run_agent_async("what is the per litre petrol price today in Mumbai. Divide the per litre price by 85")

In [ ]:
await run_agent_async("what is the per litre petrol price today in Mumbai. Divide the per litre price by 85. What do you think about the future of petrol cars?")

### Explain the steps to get: SerpAPI Key

Here's a **simple step-by-step guide** to get your **SerpAPI Key** (used for Google Search via API):

---

## What is SerpAPI?

**SerpAPI** is a service that lets you access **Google Search results programmatically** using an API. It's useful for real-time search tasks in AI agents.

---

## Steps to Get Your SerpAPI Key

### **1. Go to SerpAPI's Website**
- Open: [https://serpapi.com](https://serpapi.com)

---

### **2. Sign Up (Free Plan Available)**
- Click on **"Sign Up"** (top right).
- You can sign up using:
  - Google
  - GitHub
  - Email and password

---

### **3. Verify Your Email**
- After signup, check your inbox and click the **verification link** from SerpAPI.

---

### **4. Go to Your Account Dashboard**
- After logging in, you will be redirected to:
  [https://serpapi.com/dashboard](https://serpapi.com/dashboard)

---

### **5. Copy Your API Key**
- On the dashboard homepage, you’ll see:
  - **Your Account Email**
  - **Your Private API Key** (looks like `xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx`)
- Click **"Copy"** beside the key.

---

### **6. Paste the Key in `.env` File**
In your project folder, open or create a `.env` file and paste:

```env
SERPAPI_API_KEY=your-copied-key-here
```

Example:
```env
SERPAPI_API_KEY=86e5c04ab2341dcb01de4fc34b33f2e7b640f9d7b00f7f45e
```

---

## Free Tier Info

SerpAPI gives you:
- **100 free searches/month** (for non-commercial use)
- More if you upgrade your plan

---



### Explain the steps to get:  Wolfram Alpha API Key


---

## What is Wolfram Alpha API?

The **Wolfram Alpha API** lets your applications query Wolfram Alpha’s powerful computational engine — perfect for answering math, science, and fact-based questions using real-world data.

---

## Steps to Get Wolfram Alpha API Key

### **1. Go to Wolfram Alpha Developer Portal**
- Visit: [https://developer.wolframalpha.com](https://developer.wolframalpha.com)

---

### **2. Sign In or Create a Wolfram Account**
- Click **"Sign In"** (top right)
- Choose to **create an account** if you don’t already have one

---

### **3. Navigate to the “My Apps” Section**
- Once logged in, go to: [https://developer.wolframalpha.com/portal/myapps/index.html](https://developer.wolframalpha.com/portal/myapps/index.html)

---

### **4. Click “+ New App”**
- Give your app a **name** (e.g., "LangChainAgent")
- Description is optional
- Click **“Create App”**

---

### **5. Copy Your AppID (API Key)**
- After creating the app, you’ll be shown your **AppID** — this is your **Wolfram Alpha API Key**
- It looks something like:  
  ```
  6JXG3L-K7E9K64T6A
  ```

---

### **6. Add the API Key to Your `.env` File**
In your project folder, open or create a `.env` file and paste this line:

```env
WOLFRAM_ALPHA_APPID=your-copied-appid-here
```

Example:
```env
WOLFRAM_ALPHA_APPID=6JXG3L-K7E9K64T6A
```

---

## Free Tier Info

Wolfram Alpha’s **free plan** provides:
- Around **2,000 API calls/month** for non-commercial use
- Paid plans available for higher usage or advanced features

---



# Happy Learning